In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
import pickle

In [9]:
#data_preprocessing

def data_preprocessing(dataset:pd.DataFrame):
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [10]:
# FeatureEngineering

def FeatureEngineering(dataset:pd.DataFrame, categorical_variables:list, numerical_variables:list, isTrain:bool):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()

    df = df[numerical_important + categorical_important]

    # Dummy variables
    if isTrain:
        def ohe(dataset:pd.DataFrame, categorical_columns:str):
            df = dataset.copy()
            df_info = pd.DataFrame(columns=['col', 'uniques'])
            for col in categorical_columns:
                list_Uniques = df[col].unique()
                df_info.loc[len(df_info)] = [col, ','.join(map(str, list_Uniques))]
                for name in list_Uniques:
                    df[str(col)+'_'+str(name)] = df[col].apply(lambda x : 1 if x==name else 0)
                del df[col]

            df_info.to_csv('assets/ohe_info.csv', index=False)
            return df

        df=ohe(df, categorical_important)
    else:
        df_info = pd.read_csv('assets/ohe_info.csv')
        for i in range(len(df_info)):
            col = df_info.iloc[i,:].col
            list_Uniques = df_info.iloc[i,:].uniques.split(',')
            for name in list_Uniques:
                df[str(col)+'_'+str(name)] =  df[col].apply(lambda x : 1 if x==name else 0)
            del df[col]

    # Feature scaling
    if isTrain:
        scaler = StandardScaler()
        scaler.fit(df)
        df = scaler.transform(df)
        df = pd.DataFrame(df, columns=scaler.feature_names_in_)
        pickle.dump(scaler, open('assets/scaler.pkl','wb'))
    else:
        scaler = pickle.load(open('assets/scaler.pkl','rb'))
        df = scaler.transform(df)
        df = pd.DataFrame(df, columns=scaler.feature_names_in_)

    return df

In [11]:
df = pd.read_csv('Churn_prediction.csv')
df.drop_duplicates(inplace=True)

<h2>Split dataset</h2>

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='Churn'],
                                                    df['Churn'],
                                                    test_size=0.2,
                                                    random_state=1)

X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)

y_train = [1 if x == 'Yes' else 0 for x in y_train]
y_test = [1 if x == 'Yes' else 0 for x in y_test]

In [13]:
categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

X_train = data_preprocessing(X_train)
X_train = FeatureEngineering(X_train,
                              categorical_variables=categorical_important,
                              numerical_variables=numerical_important,
                              isTrain=True)


<h2>1) Logistic Regression</h2>

In [14]:
from random import shuffle
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
clf_LogisticRegression = LogisticRegression(random_state=0)


scores = cross_validate(clf_LogisticRegression, X_train, y_train, cv=10,
                        scoring=('roc_auc'),
                        return_train_score=True)
# scores['test_score']
print(scores['test_score'].mean())
scores



0.8376264239962852


{'fit_time': array([0.08399796, 0.05193305, 0.0551672 , 0.0739975 , 0.07336783,
        0.06694198, 0.06601596, 0.07080531, 0.04302287, 0.03884459]),
 'score_time': array([0.00893831, 0.00701499, 0.01686835, 0.00699735, 0.00696564,
        0.00507402, 0.0070076 , 0.00500035, 0.00699186, 0.0060041 ]),
 'test_score': array([0.85365036, 0.83771398, 0.83990164, 0.81330407, 0.81135549,
        0.86550775, 0.83918395, 0.86615604, 0.81463696, 0.83485401]),
 'train_score': array([0.83714863, 0.83912527, 0.83828389, 0.84222364, 0.84223135,
        0.83614093, 0.83873838, 0.83602057, 0.84135448, 0.83923611])}

<h2>2) K Nearest Neighbors</h2>

In [15]:
from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier()

scores = cross_validate(clf_KNN, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())


0.790584485844429


<h2>3) Naive Bayes</h2>

In [16]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()

scores = cross_validate(clf_NB, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())


0.8249552305843864


<h2>4) Support Vector machine for Classification (SVC)</h2>

In [17]:
from sklearn.svm import SVC
clf_SVC = SVC(kernel='linear', random_state=6)

scores = cross_validate(clf_SVC, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# kernel
#  -sigmoid   0.7217249049598338
#  -poly      0.8158020900887415
#  -rbf       0.7720218063211098
#  -linear    0.8092300188309788


0.8122607028139761


<h2>5) Desicion Tree Classification</h2>

In [18]:
from calendar import c
from sklearn.tree import DecisionTreeClassifier
clf_DT = DecisionTreeClassifier(criterion='entropy', random_state=6)

scores = cross_validate(clf_DT, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# criterion
#   -gini      0.666598842172868
#   -entropy   0.6673704556237601

0.6613870088676238


<h2>6) Random Forest</h2>

In [19]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=6)

scores = cross_validate(clf_RF, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# criterion
#   -gini       0.7940777923624925    
#   -entropy    0.7955202892868366  

0.7972289630146374


<h2>7) XGBoost</h2>

In [20]:
from xgboost import XGBClassifier

clf_xgboost = XGBClassifier()

scores = cross_validate(clf_xgboost, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())

0.8149869204235021


<h2>8) Gradiant Boosting model</h2>

In [21]:
from sklearn.ensemble import GradientBoostingClassifier

clf_GB = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3)

scores = cross_validate(clf_GB, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())

0.8409460200412692
